In [1]:
import json
import numpy
import os
import csv
import pandas as pd
import geojson
from geojson import Feature, Point, FeatureCollection
from collections import OrderedDict, defaultdict

In [2]:
# Import Dependencies
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func

In [3]:
# Load initial dataset

file_to_load = 'data/attacks.csv'

shark_data = pd.read_csv(file_to_load)
shark_data.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order
0,2017.06.11,11-Jun-17,2017.0,Unprovoked,AUSTRALIA,Western Australia,"Point Casuarina, Bunbury",Body boarding,Paul Goff,M,...,N,08h30,"White shark, 4 m","WA Today, 6/11/2017",2017.06.11-Goff.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.06.11,2017.06.11,6095.0
1,2017.06.10.b,10-Jun-17,2017.0,Unprovoked,AUSTRALIA,Victoria,"Flinders, Mornington Penisula",Surfing,female,F,...,N,15h45,7 gill shark,NaN,2017.06.10.b-Flinders.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.06.10.b,2017.06.10.b,6094.0
2,2017.06.10.a,10-Jun-17,2017.0,Unprovoked,USA,Florida,"Ponce Inlet, Volusia County",Surfing,Bryan Brock,M,...,N,10h00,NaN,"Daytona Beach News-Journal, 6/10/2017",2017.06.10.a-Brock.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.06.10.a,2017.06.10.a,6093.0
3,2017.06.07.R,Reported 07-Jun-2017,2017.0,Unprovoked,UNITED KINGDOM,South Devon,Bantham Beach,Surfing,Rich Thomson,M,...,N,NaN,"3m shark, probably a smooth hound","C. Moore, GSAF",2017.06.07.R-Thomson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.06.07.R,2017.06.07.R,6092.0
4,2017.06.04,04-Jun-17,2017.0,Unprovoked,USA,Florida,"Middle Sambo Reef off Boca Chica, Monroe County",Spearfishing,Parker Simpson,M,...,N,NaN,8' shark,"Nine News, 6/7/2017",2017.06.04-Simpson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.06.04,2017.06.04,6091.0


# Isolate Florida

In [4]:
florida_data = shark_data[shark_data['Area']=='Florida']

florida_data.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order
2,2017.06.10.a,10-Jun-17,2017.0,Unprovoked,USA,Florida,"Ponce Inlet, Volusia County",Surfing,Bryan Brock,M,...,N,10h00,NaN,"Daytona Beach News-Journal, 6/10/2017",2017.06.10.a-Brock.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.06.10.a,2017.06.10.a,6093.0
4,2017.06.04,04-Jun-17,2017.0,Unprovoked,USA,Florida,"Middle Sambo Reef off Boca Chica, Monroe County",Spearfishing,Parker Simpson,M,...,N,NaN,8' shark,"Nine News, 6/7/2017",2017.06.04-Simpson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.06.04,2017.06.04,6091.0
7,2017.05.28,28-May-17,2017.0,Unprovoked,USA,Florida,Off Jupiter,Feeding sharks,Randy Jordan,M,...,N,Morning,Tiger shark,"M. Michaelson, GSAF",2017.05.28-Jordan.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.05.28,2017.05.28,6088.0
18,2017.04.26,26-Apr-17,2017.0,Invalid,USA,Florida,NaN,Photo shoot,Molly Cavelli,F,...,NaN,NaN,No shark invovlement - it ws a publicity stunt,"The Sun, 5/6/2017",2017.05.26-Cavelli.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.04.26,2017.04.26,6077.0
21,2017.04.17.b,17-Apr-17,2017.0,Unprovoked,USA,Florida,"Daytona Beach, Volusia County",NaN,NaN,NaN,...,N,Afternoon,NaN,"Daytona Beach News-Journal, 4/17/2017",2017.04.17.b-Volusia.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.04.17.b,2017.04.17.b,6074.0


In [5]:
# create 'new_county' column

florida_data['new_county']= florida_data['Location'].str.split(',').str[-1]
florida_data.head()

<ipython-input-5-99679f1ebaba>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  florida_data['new_county']= florida_data['Location'].str.split(',').str[-1]


,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,new_county
2,2017.06.10.a,10-Jun-17,2017.0,Unprovoked,USA,Florida,"Ponce Inlet, Volusia County",Surfing,Bryan Brock,M,...,10h00,NaN,"Daytona Beach News-Journal, 6/10/2017",2017.06.10.a-Brock.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.06.10.a,2017.06.10.a,6093.0,Volusia County
4,2017.06.04,04-Jun-17,2017.0,Unprovoked,USA,Florida,"Middle Sambo Reef off Boca Chica, Monroe County",Spearfishing,Parker Simpson,M,...,NaN,8' shark,"Nine News, 6/7/2017",2017.06.04-Simpson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.06.04,2017.06.04,6091.0,Monroe County
7,2017.05.28,28-May-17,2017.0,Unprovoked,USA,Florida,Off Jupiter,Feeding sharks,Randy Jordan,M,...,Morning,Tiger shark,"M. Michaelson, GSAF",2017.05.28-Jordan.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.05.28,2017.05.28,6088.0,Off Jupiter
18,2017.04.26,26-Apr-17,2017.0,Invalid,USA,Florida,NaN,Photo shoot,Molly Cavelli,F,...,NaN,No shark invovlement - it ws a publicity stunt,"The Sun, 5/6/2017",2017.05.26-Cavelli.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.04.26,2017.04.26,6077.0,NaN
21,2017.04.17.b,17-Apr-17,2017.0,Unprovoked,USA,Florida,"Daytona Beach, Volusia County",NaN,NaN,NaN,...,Afternoon,NaN,"Daytona Beach News-Journal, 4/17/2017",2017.04.17.b-Volusia.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.04.17.b,2017.04.17.b,6074.0,Volusia County


In [6]:
# export csv

florida_data.to_csv("output_data/florida_unclean_counties.csv", index=False, header=True)

# Load cleaned counties

removed a few spaces in the 'new_county' column

A few counties were showing up as duplicates because there was a space in front of their name

In [7]:
load = 'output_data/florida_clean_counties.csv'

county_clean = pd.read_csv(load)
county_clean.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,new_county
0,2017.06.10.a,10-Jun-17,2017.0,Unprovoked,USA,Florida,"Ponce Inlet, Volusia County",Surfing,Bryan Brock,M,...,10h00,NaN,"Daytona Beach News-Journal, 6/10/2017",2017.06.10.a-Brock.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.06.10.a,2017.06.10.a,6093.0,Volusia County
1,2017.06.04,4-Jun-17,2017.0,Unprovoked,USA,Florida,"Middle Sambo Reef off Boca Chica, Monroe County",Spearfishing,Parker Simpson,M,...,NaN,8' shark,"Nine News, 6/7/2017",2017.06.04-Simpson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.06.04,2017.06.04,6091.0,Monroe County
2,2017.05.28,28-May-17,2017.0,Unprovoked,USA,Florida,Off Jupiter,Feeding sharks,Randy Jordan,M,...,Morning,Tiger shark,"M. Michaelson, GSAF",2017.05.28-Jordan.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.05.28,2017.05.28,6088.0,Palm Beach County
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017.04.17.b,17-Apr-17,2017.0,Unprovoked,USA,Florida,"Daytona Beach, Volusia County",NaN,NaN,NaN,...,Afternoon,NaN,"Daytona Beach News-Journal, 4/17/2017",2017.04.17.b-Volusia.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.04.17.b,2017.04.17.b,6074.0,Volusia County


In [8]:
county_clean.Type.value_counts()

Unprovoked      826
Provoked         79
Invalid          59
Boat              6
Boating           2
Sea Disaster      1
Name: Type, dtype: int64

In [9]:
county_clean.Activity.value_counts().reset_index()

,index,Activity
0,Surfing,356
1,Swimming,128
2,Wading,66
3,Standing,32
4,Fishing,30
...,...,...
172,"S.S. Marine Sulphur Queen, laden with molten s...",1
173,Water-skiing,1
174,Removing shark from a net,1
175,Wading with surfboard,1


# Get total attacks and fatal/nonfatal

In [10]:
fatal_by_county = pd.crosstab(index=county_clean['new_county'], columns=county_clean['Fatal (Y/N)'], margins=True)

In [11]:
# Rename columns

fatal_by_county = fatal_by_county.rename(columns={'N':'Nonfatal', 'Y': 'Fatal', 'All': 'Attacks'})

In [12]:
# reset index
fatal_by_county.reset_index(inplace=True)

fatal_by_county.head()

Fatal (Y/N),new_county,Nonfatal,UNKNOWN,Fatal,Attacks
0,Bay County,11,0,2,13
1,Brevard County,103,0,1,104
2,Broward County,21,0,2,23
3,Charlotte County,1,0,1,2
4,Citrus County,0,0,1,1


In [13]:
fatal_by_county['new_county'] = fatal_by_county['new_county'].str.replace('Dade','Miami-Dade')

# Fatal by County

In [14]:
fatal_by_county

Fatal (Y/N),new_county,Nonfatal,UNKNOWN,Fatal,Attacks
0,Bay County,11,0,2,13
1,Brevard County,103,0,1,104
2,Broward County,21,0,2,23
3,Charlotte County,1,0,1,2
4,Citrus County,0,0,1,1
5,Collier County,8,0,0,8
6,Miami-Dade County,30,1,4,35
7,Duval County,39,0,3,42
8,Escambia County,8,1,9,18
9,Flagler County,6,0,0,6


# Load in GEO data

In [15]:
load_county = 'data/counties.csv'

county_data = pd.read_csv(load_county)
county_data.head()


,Geo Point,Geo Shape,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,STUSAB,LSAD,...,CSAFP,CBSAFP,METDIVFP,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,STATE_NAME,COUNTYFP NOZERO
0,"28.5144162588, -81.3234735281","{""type"": ""Polygon"", ""coordinates"": [[[-81.6573...",12,95,295750,12095,Orange,Orange County,FL,6,...,422.0,36740.0,NaN,A,2335820971,262462219,28.514391,-81.323284,Florida,95
1,"29.2102022904, -82.0566756823","{""type"": ""Polygon"", ""coordinates"": [[[-82.4045...",12,83,306922,12083,Marion,Marion County,FL,6,...,NaN,36100.0,NaN,A,4113978836,192297049,29.202805,-82.043100,Florida,83
2,"29.7258227162, -82.8003665306","{""type"": ""Polygon"", ""coordinates"": [[[-82.9590...",12,41,303633,12041,Gilchrist,Gilchrist County,FL,6,...,264.0,23540.0,NaN,A,905705573,14362143,29.723456,-82.795801,Florida,41
3,"26.9564459351, -81.1890293853","{""type"": ""Polygon"", ""coordinates"": [[[-81.2673...",12,43,295743,12043,Glades,Glades County,FL,6,...,NaN,NaN,NaN,A,2089001842,466834350,26.954810,-81.190820,Florida,43
4,"29.2822055783, -82.7886132018","{""type"": ""Polygon"", ""coordinates"": [[[-83.0486...",12,75,295724,12075,Levy,Levy County,FL,6,...,264.0,23540.0,NaN,A,2896183010,762935040,29.284457,-82.783462,Florida,75


In [16]:
# drop unwanted columns

county_data.drop(columns=['INTPTLAT','INTPTLON','Geo Point','STATEFP','COUNTYFP','COUNTYNS','GEOID','NAME','STUSAB','CSAFP','METDIVFP','FUNCSTAT','ALAND','AWATER','STATE_NAME','COUNTYFP NOZERO','LSAD','CLASSFP','MTFCC','CBSAFP'],inplace=True)
county_data

,Geo Shape,NAMELSAD
0,"{""type"": ""Polygon"", ""coordinates"": [[[-81.6573...",Orange County
1,"{""type"": ""Polygon"", ""coordinates"": [[[-82.4045...",Marion County
2,"{""type"": ""Polygon"", ""coordinates"": [[[-82.9590...",Gilchrist County
3,"{""type"": ""Polygon"", ""coordinates"": [[[-81.2673...",Glades County
4,"{""type"": ""Polygon"", ""coordinates"": [[[-83.0486...",Levy County
...,...,...
62,"{""type"": ""Polygon"", ""coordinates"": [[[-84.7817...",Liberty County
63,"{""type"": ""Polygon"", ""coordinates"": [[[-81.8290...",DeSoto County
64,"{""type"": ""Polygon"", ""coordinates"": [[[-81.3464...",Osceola County
65,"{""type"": ""Polygon"", ""coordinates"": [[[-84.5043...",Wakulla County


# Merge Attack data with GeoJson data

In [17]:
# Merge datasets with a left join

final_geo_data = pd.merge(county_data, fatal_by_county, left_on='NAMELSAD', right_on='new_county', how = 'left')

#final_data.Year=final_data.Year.astype('int')

final_geo_data

,Geo Shape,NAMELSAD,new_county,Nonfatal,UNKNOWN,Fatal,Attacks
0,"{""type"": ""Polygon"", ""coordinates"": [[[-81.6573...",Orange County,Orange County,1.0,0.0,0.0,1.0
1,"{""type"": ""Polygon"", ""coordinates"": [[[-82.4045...",Marion County,NaN,NaN,NaN,NaN,NaN
2,"{""type"": ""Polygon"", ""coordinates"": [[[-82.9590...",Gilchrist County,NaN,NaN,NaN,NaN,NaN
3,"{""type"": ""Polygon"", ""coordinates"": [[[-81.2673...",Glades County,NaN,NaN,NaN,NaN,NaN
4,"{""type"": ""Polygon"", ""coordinates"": [[[-83.0486...",Levy County,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
62,"{""type"": ""Polygon"", ""coordinates"": [[[-84.7817...",Liberty County,NaN,NaN,NaN,NaN,NaN
63,"{""type"": ""Polygon"", ""coordinates"": [[[-81.8290...",DeSoto County,NaN,NaN,NaN,NaN,NaN
64,"{""type"": ""Polygon"", ""coordinates"": [[[-81.3464...",Osceola County,NaN,NaN,NaN,NaN,NaN
65,"{""type"": ""Polygon"", ""coordinates"": [[[-84.5043...",Wakulla County,Wakulla County,2.0,0.0,0.0,2.0


In [18]:
# drop 'new county' column

final_geo_data.drop(columns=['new_county'],inplace=True)
final_geo_data.head()

,Geo Shape,NAMELSAD,Nonfatal,UNKNOWN,Fatal,Attacks
0,"{""type"": ""Polygon"", ""coordinates"": [[[-81.6573...",Orange County,1.0,0.0,0.0,1.0
1,"{""type"": ""Polygon"", ""coordinates"": [[[-82.4045...",Marion County,NaN,NaN,NaN,NaN
2,"{""type"": ""Polygon"", ""coordinates"": [[[-82.9590...",Gilchrist County,NaN,NaN,NaN,NaN
3,"{""type"": ""Polygon"", ""coordinates"": [[[-81.2673...",Glades County,NaN,NaN,NaN,NaN
4,"{""type"": ""Polygon"", ""coordinates"": [[[-83.0486...",Levy County,NaN,NaN,NaN,NaN


In [19]:
# replace 'NaN' with 0  (these counties had zero shark attacks)

final_geo_data['Attacks'] = final_geo_data['Attacks'].fillna(0)
final_geo_data['Fatal'] = final_geo_data['Fatal'].fillna(0)
final_geo_data['UNKNOWN'] = final_geo_data['UNKNOWN'].fillna(0)
final_geo_data['Nonfatal'] = final_geo_data['Nonfatal'].fillna(0)

In [20]:
final_geo_data

,Geo Shape,NAMELSAD,Nonfatal,UNKNOWN,Fatal,Attacks
0,"{""type"": ""Polygon"", ""coordinates"": [[[-81.6573...",Orange County,1.0,0.0,0.0,1.0
1,"{""type"": ""Polygon"", ""coordinates"": [[[-82.4045...",Marion County,0.0,0.0,0.0,0.0
2,"{""type"": ""Polygon"", ""coordinates"": [[[-82.9590...",Gilchrist County,0.0,0.0,0.0,0.0
3,"{""type"": ""Polygon"", ""coordinates"": [[[-81.2673...",Glades County,0.0,0.0,0.0,0.0
4,"{""type"": ""Polygon"", ""coordinates"": [[[-83.0486...",Levy County,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...
62,"{""type"": ""Polygon"", ""coordinates"": [[[-84.7817...",Liberty County,0.0,0.0,0.0,0.0
63,"{""type"": ""Polygon"", ""coordinates"": [[[-81.8290...",DeSoto County,0.0,0.0,0.0,0.0
64,"{""type"": ""Polygon"", ""coordinates"": [[[-81.3464...",Osceola County,0.0,0.0,0.0,0.0
65,"{""type"": ""Polygon"", ""coordinates"": [[[-84.5043...",Wakulla County,2.0,0.0,0.0,2.0


# Add coast data

In [21]:
coasts = ['Inland','Inland','Inland','Inland', 'West',
         'Inland', 'Inland', 'Inland', 'Inland','West',
          'East', 'West','East','Inland', 'Inland',
          'Inland','Inland','Inland','West',
          'East','West','West','West','Inland',
          'East','East', 'Inland','West', 'Inland',

          'Inland', 'Inland', 'West', 'Inland', 'East',
          'West', 'West','West','West','West',
          'South', 'Inland','East','Inland','Inland',
          'Inland','East','East','Inland','West',
          'Inland','East','West','West','West',
          'West','Inland','Inland','West','Inland',
          
          'Inland','West','East','Inland','Inland',
          'Inland', 'West','East'         
         
         ]

In [22]:
# add coast column
final_geo_data['Coast'] = coasts

In [23]:
# Rename columns

final_geo_data = final_geo_data.rename(columns={'NAMELSAD':'County', 'UNKNOWN': 'Unknown', 'Geo Shape': 'Geo_shape'})

In [24]:
# Reorder columns

column_names = ["County", "Attacks", "Fatal","Nonfatal","Unknown","Coast","Geo_shape"]

final_geo_data = final_geo_data.reindex(columns=column_names)

In [25]:
# change float to integers

final_geo_data = final_geo_data.astype({"Attacks":'int', "Fatal":'int',"Nonfatal":'int',"Unknown":'int'}) 
  

In [26]:
# displaying the datatypes
display(final_geo_data.dtypes)

County       object
Attacks       int32
Fatal         int32
Nonfatal      int32
Unknown       int32
Coast        object
Geo_shape    object
dtype: object

# Final Dataframe

In [27]:
final_geo_data

,County,Attacks,Fatal,Nonfatal,Unknown,Coast,Geo_shape
0,Orange County,1,0,1,0,Inland,"{""type"": ""Polygon"", ""coordinates"": [[[-81.6573..."
1,Marion County,0,0,0,0,Inland,"{""type"": ""Polygon"", ""coordinates"": [[[-82.4045..."
2,Gilchrist County,0,0,0,0,Inland,"{""type"": ""Polygon"", ""coordinates"": [[[-82.9590..."
3,Glades County,0,0,0,0,Inland,"{""type"": ""Polygon"", ""coordinates"": [[[-81.2673..."
4,Levy County,0,0,0,0,West,"{""type"": ""Polygon"", ""coordinates"": [[[-83.0486..."
...,...,...,...,...,...,...,...
62,Liberty County,0,0,0,0,Inland,"{""type"": ""Polygon"", ""coordinates"": [[[-84.7817..."
63,DeSoto County,0,0,0,0,Inland,"{""type"": ""Polygon"", ""coordinates"": [[[-81.8290..."
64,Osceola County,0,0,0,0,Inland,"{""type"": ""Polygon"", ""coordinates"": [[[-81.3464..."
65,Wakulla County,2,0,2,0,West,"{""type"": ""Polygon"", ""coordinates"": [[[-84.5043..."


In [28]:
# export csv

final_geo_data.to_csv("output_data/final_data.csv", index=False, header=True)

# Dataframe for pie chart

In [45]:
pie = final_geo_data.aggregate(['sum'])
pie

,County,Attacks,Fatal,Nonfatal,Unknown,Coast,Geo_shape
sum,Orange CountyMarion CountyGilchrist CountyGlad...,973,57,906,10,InlandInlandInlandInlandWestInlandInlandInland...,"{""type"": ""Polygon"", ""coordinates"": [[[-81.6573..."


In [46]:
# Drop columns

pie_final = pie.drop(['County', 'Geo_shape', 'Coast'], axis = 1)

In [47]:
# export csv

pie_final.to_csv("output_data/data_for_pie.csv", index=False, header=True)

# Dataframe for bar chart

In [32]:
bar = final_geo_data.groupby('Coast').aggregate('sum')
bar

,Attacks,Fatal,Nonfatal,Unknown
Coast,,,,
East,790,28,754,8
Inland,1,0,1,0
South,60,8,51,1
West,122,21,100,1


In [48]:
# export csv

bar.to_csv("output_data/data_for_bar.csv", index=True, header=True)

# Create GeoJSON

In [34]:
final_geo = xfinal_geo_data.to_dict(orient='records',)

NameError: name 'xfinal_geo_data' is not defined

In [ ]:
final_geo2 = final_geo_data.to_dict(orient='records',into=OrderedDict)

In [ ]:
x = final_geo2[0]['Geo_shape']

In [ ]:
y = json.loads(x)

In [ ]:
# write dict to json and export
     
with open("sample.json", "w") as outfile: 
    json.dump(final_geo, outfile)


In [ ]:
florida = jsonify({"type":"FeatureCollection", "features": final_geo_data["Geo Shape"].to_list()})

In [38]:
load_json = 'data/us-county-boundaries.geojson'

import json
  
# Opening JSON file
f = open('data/us-county-boundaries.geojson',)
  
# returns JSON object as 
# a dictionary
data = json.load(f)
  

In [41]:
data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-81.65739, 28.553122],
      [-81.657394, 28.553157],
      [-81.657397, 28.553178],
      [-81.657393, 28.553263],
      [-81.657392, 28.553299],
      [-81.657392, 28.553309],
      [-81.657373, 28.554039],
      [-81.657344, 28.555155],
      [-81.657314, 28.556261],
      [-81.657295, 28.557002],
      [-81.657096, 28.562778],
      [-81.657074, 28.563228],
      [-81.657041, 28.563938],
      [-81.656772, 28.569709],
      [-81.656523, 28.569698],
      [-81.656417, 28.569709],
      [-81.656349, 28.569758],
      [-81.65623, 28.569896],
      [-81.655918, 28.570154],
      [-81.655632, 28.570402],
      [-81.65547, 28.570462],
      [-81.655046, 28.570545],
      [-81.654235, 28.570665],
      [-81.653313, 28.570896],
      [-81.652877, 28.570946],
      [-81.652771, 28.570995],
      [-81.652665, 28.571144],
      [-81.652652, 28.57203],
      [-81.652222, 28.

In [ ]:
# Opening JSON file
f = open('data/sample.json',)
  
# returns JSON object as 
# a dictionary
data2 = json.load(f)

In [ ]:
data_shark = data2[0]



with open("test_shark.json", "w") as outfile: 
    json.dump(data_shark, outfile)




data2[0]

In [ ]:
with open('data.json','w') as f:
    json.dump(dict_in_geojson_form, f)
